# Uncertainty quantification for cross sections
This tutorial will demonstrate how to use jitr to perform uncertainty quantification of a simple reaction observable: differential elastic scattering cross sections. We will use the uncertainty quantified optical potential KDUQ introduced here: [[Pruitt et al., 2024]](https://journals.aps.org/prc/abstract/10.1103/PhysRevC.107.014602). Jitr includes an implementation of the kduq potential, as well as a set of samples of the posterior provided in the supplemental material of that paper.

In [1]:
from pathlib import Path

import corner

# import stuff for nice plotting
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.lines import Line2D
from numba import njit
from scipy import stats
from tqdm import tqdm

import jitr

We will use [exfor-tools](https://github.com/beykyle/exfor_tools) using an [x4i3](https://github.com/afedynitch/x4i3/) backend for grabbing data from [EXFOR](https://www-nds.iaea.org/exfor/) (requires 1GB of disk space to download).

In [2]:
import exfor_tools

Using database version x4i3_X4-2023-04-29 located in: /home/beyerk/mambaforge/envs/sage/lib/python3.11/site-packages/x4i3/data


### Let's find some data to compare to
Let's grab some data from [EXFOR](https://www-nds.iaea.org/exfor/). We will look at $d\sigma_{el}/d\Omega$ for $(n,n)$ and $(p,p)$.

In [3]:
# target to consider
Pb208 = (208, 82)
proton = (1, 1)
neutron = (1, 0)

# for plotting differential xs
angles = np.linspace(0.1, np.pi, 100)

In [4]:
def interaction_scalar(r, kd_scalar_params, coulomb_params):
    kd = jitr.reactions.kduq.KD_scalar(r, *kd_scalar_params)
    coul = jitr.reactions.potentials.coulomb_charged_sphere(r, *coulomb_params)
    return kd + coul

In [5]:
sys = jitr.reactions.ProjectileTargetSystem(
    channel_radius=10 * np.pi,
    lmax=15,
    mass_target=jitr.utils.kinematics.mass(*Pb208),
    mass_projectile=jitr.utils.kinematics.mass(*neutron),
    Ztarget=Pb208[1],
    Zproj=0,
    coupling=jitr.reactions.system.spin_half_orbit_coupling,
)

In [6]:
Ecm = 12
mu, Elab, k, eta = jitr.utils.kinematics.classical_kinematics_cm(
    sys.mass_target, sys.mass_projectile, Ecm, sys.Zproj * sys.Ztarget
)

In [7]:
ws = jitr.xs.ElasticXSWorkspace(
    projectile=neutron,
    target=Pb208,
    sys=sys,
    Ecm=Ecm,
    k=k,
    mu=mu,
    eta=eta,
    local_interaction_scalar=interaction_scalar,
    local_interaction_spin_orbit=jitr.reactions.KD_spin_orbit,
    solver=jitr.rmatrix.Solver(50),
    angles=angles,
)

In [10]:
# we have 416 samples from the KDUQ posterior
num_samples_kduq = 415
param_files = [
    Path(f"./../../src/data/KDUQFederal/{i}/parameters.json")
    for i in range(0, num_samples_kduq)
]

# load each one
kduq_omps = [
    rose.koning_delaroche.KDGlobal(rose.Projectile.neutron, Path(param_file))
    for param_file in param_files
]
# load each one
jitr_omps = [
    jitr.reactions.kduq.KDGlobal((1, 0), Path(param_file)) for param_file in param_files
]

In [ ]:
kduq_rose_solver.dsdo()

In [ ]:
all_entries = exfor_tools.get_exfor_differential_data(
    target=Pb208,
    projectile=(1, 0),
    quantity="dXS/dA",
    product="EL",
    energy_range=[7, 20],  # MeV
)
all_measurements = exfor_tools.sort_measurements_by_energy(all_entries)
print(
    f"Found measurements at {len(list(all_entries))} different energies for (n,n) on 208-Pb"
)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 12))
exfor_keys = list(all_entries.keys())
all_entries[exfor_keys[0]].plot_experiment(
    ax,
    all_measurements,
    offsets=50,
    label_offset_factor=2,
    label_hloc_deg=150,
    label_energy_err=False,
    label_offset=False,
)

### Let's set up our solver to calculate $^{208}$Pb $(n,n)$ differential cross sections

We will use the the KDUQ potential from [Pruitt et al., 2024](https://journals.aps.org/prc/abstract/10.1103/PhysRevC.107.014602) potential. The supplemental material in that link contains a set of samples from the posterior of the KDUQ parameter distribution. Let's load them up:

We will create an instance of a `rose.InteractionSpace` and a `rose.ScatteringAmplitudeEmulator` for each of these potentials, and we will use the high-fidelity solver to calculate the cross sections predicted by each model.

We will use a high-fidelity solver from the [jitr package](https://github.com/beykyle/jitr) which uses the R-Matrix method on a Lagrange-Legendre mesh.

In [ ]:
core_solver = jitr.rmatrix.Solver(40)

In [ ]:
energies = []
kduq_solvers = []

for measurement in all_measurements:
    Ecm = measurement.Ecm
    dEcm = measurement.dEcm
    energies.append(Ecm)

    # get kinematics and parameters for this experiment
    mu, Elab, k, eta = rose.utility.kinematics(Pb208, (1, 0), E_com=Ecm)

    kduq = rose.InteractionSpace(
        coordinate_space_potential=rose.koning_delaroche.KD_simple,
        n_theta=rose.koning_delaroche.NUM_PARAMS,
        is_complex=True,
        spin_orbit_term=rose.koning_delaroche.KD_simple_so,
        energy=Ecm,
        mu=mu,
        k=k,
        l_max=15,
    )
    rmatrix_solver = rose.LagrangeRmatrix(
        kduq.interactions[0][0],
        np.pi * 10,
        core_solver,
    )
    kduq_solvers.append(
        rose.ScatteringAmplitudeEmulator.HIFI_solver(
            base_solver=rmatrix_solver,
            interaction_space=kduq,
            angles=angles,
        )
    )

In [ ]:
num_samples_pred_post = 100
kduq_pred_post = []

for i, measurement in enumerate(all_measurements):
    Ecm = measurement.Ecm
    dEcm = measurement.dEcm
    print(f"Running UQ  at {Ecm} MeV")

    # get kinematics and parameters for this experiment
    mu, Elab, k, eta = rose.utility.kinematics(Pb208, (1, 0), E_com=Ecm)

    # get a differential xs for each parameter sample
    kduq_xs = np.zeros((len(angles), num_samples_pred_post))
    samples = np.random.choice(num_samples_kduq, size=num_samples_pred_post)

    for j, sample in enumerate(tqdm(samples)):
        omp = kduq_omps[sample]
        R_C, params = omp.get_params(*Pb208, mu, Elab, k)
        kduq_xs[:, j] = kduq_solvers[i].exact_xs(params).dsdo

    # get a credible interval describing the KDUQ predictive posterior
    # for this experiment
    kduq_pred_post.append(
        (
            np.percentile(kduq_xs, 5, axis=1),
            np.percentile(kduq_xs, 95, axis=1),
        )
    )

Now that we have our model predictions, lets plot them compared to the experimental data. We will offset each energy for visibility.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 12))
exfor_keys = list(all_entries.keys())
all_entries[exfor_keys[0]].plot_experiment(
    ax,
    all_measurements,
    offsets=30,
    label_offset_factor=2,
    label_hloc_deg=150,
    label_energy_err=False,
    label_offset=True,
)
offsets = 30 ** np.arange(len(all_measurements))
for i in range(len(all_measurements)):
    # plot model
    ax.fill_between(
        angles * 180 / np.pi,
        offsets[i] * kduq_pred_post[i][0],
        offsets[i] * kduq_pred_post[i][1],
        color="#ff4500",
        alpha=0.5,
    )